<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Random Search로 Decision Tree 알고리즘 모델을 튜닝합니다.


# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/boston.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [3]:
# 상위 몇 개 행 확인
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
# 기초통계량 확인
data.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [5]:
# target 확인
target = 'medv'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**2) 학습용, 평가용 데이터 분리**

In [6]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, 
                                                    random_state=2022)

# 확인
print(x_train.shape)
print(x_test.shape)

(354, 13)
(152, 13)


# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 모델 선언시 max_depth=5, random_state=2022를 지정합니다.

In [7]:
# 불러오기
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [8]:
# 선언하기
model = DecisionTreeRegressor(max_depth=5, random_state=2022)

In [9]:
# 학습하기
model.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=5, random_state=2022)

In [10]:
# 예측하기
y_pred = model.predict(x_test)

In [11]:
# 평가하기
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2-Score:', r2_score(y_test, y_pred))

MAE: 3.2522706568198876
R2-Score: 0.7147610837328198


# 5.성능 튜닝 #1


- Random Search로 튜닝을 진행합니다. 


**1) 모델링 & 튜닝**

In [14]:
# 파라미터 선언
  # max_depth: 1~50
rand_param = {'max_depth':range(1, 51)}
rand_param

{'max_depth': range(1, 51)}

In [15]:
# 불러오기
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV

In [16]:
# 선언하기
model_dst = DecisionTreeRegressor(random_state=2022)

# Random Search 선언
  # cv=10
  # n_iter=30
  # scoring='r2'
model = RandomizedSearchCV(model_dst, # 기본 모델 이름
                          rand_param, # 앞에서 선언한 튜닝용 파라미터 변수
                          cv=10,     # K-Fold Cross Validation(default=5)
                          n_iter=30, # Random하게 시도할 횟수(default=10)
                          scoring='r2', # 평가 방법
                          n_jobs=-1) # cpu성능 좋아지게 병렬처리

In [17]:
# 학습하기
model.fit(x_train, y_train)

RandomizedSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=2022),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'max_depth': range(1, 51)},
                   scoring='r2')

**2) 결과 확인**

In [18]:
model.cv_results_

{'mean_fit_time': array([0.01174254, 0.00847023, 0.00776498, 0.00980101, 0.00741661,
        0.0114244 , 0.00944343, 0.00932794, 0.00921278, 0.00890665,
        0.00875471, 0.00999696, 0.00953774, 0.00931785, 0.00887876,
        0.00904078, 0.00996358, 0.01011124, 0.01072571, 0.00975132,
        0.00913932, 0.00940435, 0.00918431, 0.00879779, 0.00894449,
        0.00975199, 0.00922596, 0.01011279, 0.00989234, 0.00882716]),
 'std_fit_time': array([0.00328705, 0.00116001, 0.00224722, 0.00124478, 0.00119415,
        0.00331387, 0.00087364, 0.00278542, 0.00072586, 0.00086258,
        0.00056031, 0.00168443, 0.00192818, 0.00112289, 0.00093936,
        0.00122374, 0.00169988, 0.00215301, 0.00308197, 0.00207594,
        0.00081858, 0.00099262, 0.0008779 , 0.00065792, 0.00117584,
        0.00206892, 0.00086242, 0.00188463, 0.00284294, 0.00128364]),
 'mean_score_time': array([0.00480976, 0.00423908, 0.00549212, 0.00438826, 0.00410073,
        0.00421479, 0.00382245, 0.00369623, 0.00419531, 0.00

In [19]:
# mean_test_score 확인
model.cv_results_['mean_test_score']

array([0.67043127, 0.75058607, 0.64387954, 0.66904156, 0.6747913 ,
       0.67043127, 0.67043127, 0.64043612, 0.67043127, 0.64117459,
       0.68980603, 0.67043127, 0.67043127, 0.67043127, 0.67043127,
       0.69424064, 0.67043127, 0.68845066, 0.67043127, 0.67043127,
       0.67043127, 0.67043127, 0.67043127, 0.71325998, 0.66387521,
       0.67043127, 0.67170607, 0.67043127, 0.66897665, 0.67043127])

In [20]:
# 최적 파라미터
model.best_params_

{'max_depth': 6}

In [21]:
# 최적 파라미터 사용시 성능
model.best_score_

0.750586066038991

**3) 예측과 평가**

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [22]:
# 예측하기
y_pred = model.predict(x_test)

In [23]:
# 평가하기
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2-Score:', r2_score(y_test, y_pred))

MAE: 3.289997930052067
R2-Score: 0.7155124286118656


--------------------------------------------------------------------------

# 6.성능 튜닝 #2

- Grid Search로 튜닝을 진행합니다. 

**1) 모델링 & 튜닝**

- 성능을 확인할 파라미터를 딕셔너리 형태로 선언합니다.

In [57]:
# 파라미터 선언
  # max_depth: 1~50
grid_param = {'max_depth':[3, 4, 5, 6, 7, 8, 9]}
grid_param

{'max_depth': [3, 4, 5, 6, 7, 8, 9]}

In [58]:
# 불러오기
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [59]:
# 선언하기
model_dst = DecisionTreeRegressor(random_state=2022)

# Grid Search 모델 선언하기
  # cv=10
  # scoring='r2'
model = GridSearchCV(model_dst, # 기본 모델 이름
                          grid_param, # 앞에서 선언한 튜닝용 파라미터 변수
                          cv=10, # K-Fold Cross Validation(default=5)
                          scoring='r2', # 평가 방법
                          n_jobs=-1) # cpu성능 좋아지게 병렬처리

In [60]:
# 학습하기
model.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=2022),
             n_jobs=-1, param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9]},
             scoring='r2')

**2) 결과 확인**

In [61]:
model.cv_results_

{'mean_fit_time': array([0.00684927, 0.00694923, 0.00730288, 0.00840948, 0.00812836,
        0.00837667, 0.00797629]),
 'std_fit_time': array([0.00068877, 0.00117416, 0.00153265, 0.00231086, 0.00122576,
        0.00143294, 0.00123014]),
 'mean_score_time': array([0.00408399, 0.00427148, 0.00412376, 0.00368884, 0.00583444,
        0.00489862, 0.00385687]),
 'std_score_time': array([0.00083073, 0.00118177, 0.00075979, 0.00067702, 0.00250507,
        0.00241817, 0.00052701]),
 'param_max_depth': masked_array(data=[3, 4, 5, 6, 7, 8, 9],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 3},
  {'max_depth': 4},
  {'max_depth': 5},
  {'max_depth': 6},
  {'max_depth': 7},
  {'max_depth': 8},
  {'max_depth': 9}],
 'split0_test_score': array([0.7815458 , 0.84976308, 0.87111301, 0.79788356, 0.80482506,
        0.81289098, 0.83457009]),
 'split1_test_score': array([0.59193002, 0.63194801, 0.67960224, 

In [62]:
# mean_test_score 확인
model.cv_results_['mean_test_score']

array([0.6747913 , 0.70591332, 0.74343071, 0.75058607, 0.71479741,
       0.72103549, 0.71935341])

In [63]:
# 최적 파라미터
model.best_params_

{'max_depth': 6}

In [64]:
# 최적 파라미터 사용시 성능
model.best_score_

0.750586066038991

**3) 예측과 평가**

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [65]:
# 예측하기
y_pred = model.predict(x_test)

In [66]:
# 평가하기
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2-Score:', r2_score(y_test, y_pred))

MAE: 3.289997930052067
R2-Score: 0.7155124286118656
